In [4]:
import scipy
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [5]:
df = pd.read_csv("Data.csv")
df.dtypes

Direksi Perempuan      int64
Direksi Laki-Laki      int64
Proporsi Perempuan    object
Harga penutupan        int64
Total Shares           int64
Total Utang            int64
Total Asset            int64
dtype: object

In [6]:
#X = (df['Direksi Perempuan'])/(df['Direksi Perempuan'] + df['Direksi Laki-Laki'])
X = df["Direksi Perempuan"]
nilai_pasar = df["Total Shares"] * df["Harga penutupan"]
np_plus_liability = nilai_pasar + df["Total Utang"]
Y = np_plus_liability / df["Total Asset"]
#Y = nilai_pasar / df["Total Asset"]

data = pd.DataFrame({"X":X, "Y":Y})
X_mean = np.mean(data["X"])
X_std = np.std(data["X"])
Y_mean = np.mean(data["Y"])
Y_std = np.std(data["Y"])

In [7]:
data_sample = pd.DataFrame({"X": X, "Y": Y})
data_sample.describe()

,X,Y
count,155.000000,155.000000
mean,1.406452,1.285914
std,1.277664,1.987625
min,0.000000,0.159993
25%,0.500000,0.929432
50%,1.000000,0.973140
75%,2.000000,1.124461
max,6.000000,18.327231


In [8]:
# data_sample.drop(data_sample[data_sample["X"] == 0].index, inplace=True)
# np.size(np.where(data_sample["X"] == 0))

In [9]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.5772
Date:                Tue, 16 May 2023   Prob (F-statistic):              0.449
Time:                        00:12:44   Log-Likelihood:                -325.62
No. Observations:                 155   AIC:                             655.2
Df Residuals:                     153   BIC:                             661.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4201      0.238      5.962      0.000       0.950       1.891
X             -0.0954      0.126     -0.760      0.449      -0.343       0.153
==============================================================================
Omnibus:                      266.544   Durbin-Watson:                   0.888
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            30212.481
Skew:                           8.108   Prob(JB):                         0.00
Kurtosis:                      69.446   Cond. No.                         3.31
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
bX = data_sample["X"] * result.params[0]
abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [11]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.550
Model:                            OLS   Adj. R-squared:                  0.548
Method:                 Least Squares   F-statistic:                     187.3
Date:                Tue, 16 May 2023   Prob (F-statistic):           2.37e-28
Time:                        00:12:44   Log-Likelihood:                -288.16
No. Observations:                 155   AIC:                             580.3
Df Residuals:                     153   BIC:                             586.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7126      0.187      3.810      0.000       0.343       1.082
X              1.3493      0.099     13.687      0.000       1.155       1.544
==============================================================================
Omnibus:                      268.329   Durbin-Watson:                   1.127
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            31322.265
Skew:                           8.199   Prob(JB):                         0.00
Kurtosis:                      70.683   Cond. No.                         3.31
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
XQ3,XQ1 = np.percentile(data_sample["X"], [75 ,25])
XIQR = XQ3 - XQ1 

In [13]:
lower_X = XQ1 - (1.5 * XIQR)
upper_X = XQ3 + (1.5 * XIQR)

In [14]:
data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)]

70    6
71    6
72    6
Name: X, dtype: int64

In [15]:
data_sample.drop(data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)].index, inplace=True)

In [16]:
YQ3,YQ1 = np.percentile(data_sample['Y'], [75 ,25])
YIQR = YQ3 - YQ1 

In [17]:
lower_Y = YQ1 - (1.5 * YIQR)
upper_Y = YQ3 + (1.5 * YIQR)

In [18]:
data_sample.drop(data_sample["Y"][(data_sample["Y"] < lower_Y) | (data_sample["Y"] > upper_Y)].index, inplace=True)

In [19]:
data_sample.describe()

,X,Y
count,129.000000,129.000000
mean,1.465116,1.008960
std,1.111351,0.123998
min,0.000000,0.688744
25%,1.000000,0.932466
50%,1.000000,0.970944
75%,2.000000,1.058917
max,4.000000,1.385464


In [20]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9897
Date:                Tue, 16 May 2023   Prob (F-statistic):              0.322
Time:                        00:12:44   Log-Likelihood:                 87.246
No. Observations:                 129   AIC:                            -170.5
Df Residuals:                     127   BIC:                            -164.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9946      0.018     54.918      0.000       0.959       1.030
X              0.0098      0.010      0.995      0.322      -0.010       0.029
==============================================================================
Omnibus:                       19.181   Durbin-Watson:                   0.961
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               22.623
Skew:                           0.987   Prob(JB):                     1.22e-05
Kurtosis:                       3.556   Cond. No.                         3.68
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
bX = data_sample["X"] * result.params["X"]

In [22]:
# Y = a + bX + e
# e = Y - a - bX

abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [23]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.1829
Date:                Tue, 16 May 2023   Prob (F-statistic):              0.670
Time:                        00:12:45   Log-Likelihood:                 149.49
No. Observations:                 129   AIC:                            -295.0
Df Residuals:                     127   BIC:                            -289.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0929      0.011      8.315      0.000       0.071       0.115
X              0.0026      0.006      0.428      0.670      -0.009       0.015
==============================================================================
Omnibus:                       39.083   Durbin-Watson:                   1.225
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               66.174
Skew:                           1.432   Prob(JB):                     4.27e-15
Kurtosis:                       5.026   Cond. No.                         3.68
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""